In [2]:
# If cloning is needed -- i.e., if hangman.py and word counts are not already
# available in file, you can run this
!git clone https://github.com/ucsd-cse150a-w25/hw1.git
!cp hw1/hangman.py hangman.py
!cp hw1/hw1_word_counts_05.txt hw1_word_counts_05.txt

Cloning into 'hw1'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 55 (delta 26), reused 28 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 723.52 KiB | 8.22 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [3]:
from hangman import hangman_game

# Play the game by yourself
hangman_game()



  +---+
  O   |
 /|\  |
      |
     ===

Word: R A Z O R
Tried letters: A, I, O, R, S, T, U, Z
Congratulations! You guessed the word:  RAZOR


1

In [10]:
import random

def random_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Random inference for playing hangman. This should be a simple method which returns a letter
    that is NOT in letters_tried but any other letter, at random.

    Hint: use the random.choice method
    '''
    # TODO: Implement random inference code to guess hangman
    letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    guess = random.choice(letters)
    while guess in letters_tried:
        guess = random.choice(letters)

    return guess

hangman_game(random_inference)



  +---+
  O   |
 /|\  |
 / \  |
     ===

Word: _ _ _ _ _
Tried letters: C, D, K, O, Q, W
Game over! The word was:  STEPS


0

In [52]:
#from _typeshed import NoneType
def bayesian_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Bayesian inference method for playing hangman. The parameters given are as follows:

    - letters_tried (set[str]): A set of strings which consist of all the letters that have already
        been tried. For example, if 'A', 'E' has been guessed, `letters_tried = {'A', 'E'}`
    - word_pattern (list[str]): A list of single characters that describe the current guess state.
        For example, if the hangman state is _AB__, `word_pattern = ['_', 'A', 'B', '_', '_']`
    - word_counts (dict[str, int]): The word counts dictionary which contains all possible 5 letter
        words in our hangman game and their respective counts. For example, a key value pair could
        be 'AARON': 413.

    Return type: a single character string as your next best guess.
    '''
    # TODO: Implement inference code to play hangman optimally
    #set letters to all letters not yet tried
    letters = {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'}
    letters = letters - letters_tried
    print(letters)
    total_count = 0
    Pw = 0 #Prob of current word = w
    Pe = 0 #Prob of current evidence
    PEgivenW = 1 #Prob of evidence given word, either 0 or 1
    PwgivenE = 0 #Prob of word given evidence
    PLgivenW = 0 #Prob of letter given word, either 0 or 1
    PLgivenE = 0 #the prob of most likely letter given the evidence
    tempPLgivenE = 0 #temp prob of most likely letter given the evidence
    likelyL = None

    #finds total number of words in word_counts
    for word in word_counts:
        total_count += word_counts[word]

    #find denominator for Bayes, P(E)
    for word in word_counts:
      Pw = word_counts[word] / total_count
      PEgivenW = 1
      for i in range(0, 5): #checks if current word is valid
        if (word[i] in letters and word_pattern[i] == '_') or word_pattern[i] == word[i]: #word is still valid
          continue
        else: #word is invalid
          PEgivenw = 0
          break
      Pe += PEgivenW * Pw

    #find prob of most likely letter
    for letter in letters:

      for word in word_counts:
        #find prob of word given evidence via Bayes
        Pw = word_counts[word] / total_count
        PEgivenW = 1
        for i in range(0, 5): #checks if current word is valid
          if (word[i] in letters and word_pattern[i] == '_') or word_pattern[i] == word[i] #word is still valid
            continue
          else: #word is invalid
            PEgivenw = 0
            break
        PwgivenE = PEgivenW * Pw / Pe

        #find prob of letter given word, either 0 or 1
        '''PLgivenW = 0
        for i in range(0, 5):
          if word_pattern[i] == '_' and letter == word[i]:
            PLgivenW = 1
            break
        tempPLgivenE += PLgivenW * PwgivenE'''
        PLgivenW = 0
        for i in range(0, 5):
          if word_pattern[i] == '_' and letter == word[i]:
            PLgivenW = 1
            break
        tempPLgivenE += PLgivenW * PwgivenE

      #checks if PLgivenE of subsequent letter is greater than current one
      if tempPLgivenE > PLgivenE:
        PLgivenE = tempPLgivenE
        likelyL = letter

      #reset for next letter
      tempPLgivenE = 0

    return likelyL

# Run the game
hangman_game(bayesian_inference)



  +---+
  O   |
 /|\  |
 / \  |
     ===

Word: _ A _ _ R
Tried letters: A, E, I, L, N, R, S, T
Game over! The word was:  MAJOR


0

In [50]:
from os import sched_setscheduler
from typing import Optional, Callable
# from tqdm import tqdm

def benchmark(
    inference: Optional[Callable],
    num_runs: int = 50,
    seed: int = 0
) -> None:
    '''
    Benchmark method for testing out the bayesian inference method. The parameters given are:

    - inference: The function that should match the bayesian_inference() method above.
    - seed: The seed to pass into the hangman_game function.

    Return type: none.

    Print the accuracy out of num_runs. If the function throws an error, it should count as a fail.
    '''
    # TODO: Implement benchmark testing for a given function
    # Hint: use `random.seed(seed)` ONCE at the start
    # Hint: pass `interactive = False` into the hangman_game function to run faster without outputs.
    # Optional: use the `tqdm` module to keep track of the evaluation progress.
    random.seed(0)
    correct = 0
    for i in range(0, num_runs):
      if hangman_game(inference, interactive = False) == 1:
        correct += 1
    print(f"Accuracy: {correct / num_runs}")


benchmark(random_inference)
benchmark(bayesian_inference)

Accuracy: 0.0
Accuracy: 0.28
